In [ ]:
from __future__ import print_function

import os
import re
import cPickle, gzip
import progressbar

from rdkit import Chem
from rdkit.Chem import AllChem
import parser.Smipar as Smipar

In [ ]:
# data reloader

with gzip.open('data/vocab/vocab_dict.pkl.gz', 'rb') as dict_file:
    vocab_reactants, vocab_products = cPickle.load(dict_file)

with gzip.open('data/vocab/vocab_list.pkl.gz', 'rb') as list_file:
    reactants_token_list, products_token_list = cPickle.load(list_file)

In [ ]:
def cano(smiles): # canonicalize smiles by MolToSmiles function
    return Chem.MolToSmiles(Chem.MolFromSmiles(smiles)) if (smiles != '') else ''

In [ ]:
for i in range(13, 17):
    with gzip.open('gen_rxn(test)/data/all_rxns_14.pkl.gz', 'rb') as f:
        rxn_all = cPickle.load(f)

    data_length = len(rxn_all)
    print(data_length)

    bar = progressbar.ProgressBar(max_value=data_length)

    token_len = []

    with gzip.open('data/test/test_14.pkl.gz', 'wb') as training_file:

        saved_count = 0

        for i, rsmi in enumerate(rxn_all):

            reactant_list = []
            agent_list = []
            product_list = []

            if bool(re.search('5|6|7|8|9|1\d', rsmi)): continue

            try:
                split_rsmi = rsmi.split('>')
                reactants = cano(split_rsmi[0]).split('.')
                agents = cano(split_rsmi[1]).split('.')
                products = cano(split_rsmi[2]).split('.')

                for reactant in reactants:
                    reactant_list += Smipar.parser_list(reactant)
                    reactant_list += '.'
                for agent in agents:
                    agent_list += Smipar.parser_list(agent)
                    agent_list += '.'
                for product in products:
                    product_list += Smipar.parser_list(product)
                    product_list += '.'

                reactant_list.pop() # to pop last '.'
                agent_list.pop()
                product_list.pop()

                reactant_list += '>'
                reactant_list += agent_list

                token_len.append((len(reactant_list), len(product_list)))

                cPickle.dump(([reactants_token_list.index(r) for r in reactant_list],
                              [products_token_list.index(p) for p in product_list]), training_file, 2)
                saved_count += 1

            except:
                pass

            bar.update(i)

    bar.finish()

    print(saved_count)